In [6]:
import cv2
import numpy as np

### Task : 1 - 1

In [7]:
# Task 1: Reconstruct the fundamental matrix F using the 8-point algorithm

# Load the image points from "house1.png" and "house2.png"
image1 = cv2.imread("house1.png")
image2 = cv2.imread("house2.png")

In [8]:
# Select corresponding points in the images (10 points each)
points_image1 = [(170, 105), (223, 115), (211, 182), (134, 171), (175, 155),
                 (216, 157), (132, 119), (142, 173), (201, 173), (169, 116)]
points_image2 = [(76, 91), (132, 98), (118, 167), (43, 157), (84, 143),
                 (127, 144), (42, 104), (51, 161), (110, 162), (76, 104)]

In [9]:
# Convert points to homogeneous coordinates
points_image1_homogeneous = np.array([(x, y, 1) for x, y in points_image1])
points_image2_homogeneous = np.array([(x, y, 1) for x, y in points_image2])

In [10]:
# Normalize the coordinates (optional but recommended)
mean_image1 = np.mean(points_image1_homogeneous[:, :2], axis=0)
mean_image2 = np.mean(points_image2_homogeneous[:, :2], axis=0)
std_image1 = np.std(points_image1_homogeneous[:, :2])
std_image2 = np.std(points_image2_homogeneous[:, :2])

T_image1 = np.array([[1 / std_image1, 0, -mean_image1[0] / std_image1],
                     [0, 1 / std_image1, -mean_image1[1] / std_image1],
                     [0, 0, 1]])
T_image2 = np.array([[1 / std_image2, 0, -mean_image2[0] / std_image2],
                     [0, 1 / std_image2, -mean_image2[1] / std_image2],
                     [0, 0, 1]])

normalized_points_image1 = T_image1 @ points_image1_homogeneous.T
normalized_points_image2 = T_image2 @ points_image2_homogeneous.T

In [11]:
# Construct the matrix A for the 8-point algorithm
A = np.zeros((len(points_image1), 9))
for i in range(len(points_image1)):
    x1, y1, _ = normalized_points_image1[:, i]
    x2, y2, _ = normalized_points_image2[:, i]
    A[i] = [x1 * x2, x1 * y2, x1, y1 * x2, y1 * y2, y1, x2, y2, 1]

In [12]:
# Perform singular value decomposition on A
_, _, V = np.linalg.svd(A)

# Extract the fundamental matrix F from the last column of V
F = V[-1].reshape(3, 3)

In [13]:
# Enforce rank 2 constraint on F
U, S, V = np.linalg.svd(F)
S[-1] = 0
F = U @ np.diag(S) @ V

In [14]:
# Denormalize the fundamental matrix F
F = T_image2.T @ F @ T_image1

print("Fundamental Matrix F:")
print(F)

Fundamental Matrix F:
[[ 3.00289954e-05  8.55520281e-05 -2.36380409e-03]
 [-7.50298414e-05 -1.20890748e-05  8.16597365e-03]
 [-1.25595524e-02  2.66255750e-03  1.41944725e+00]]


### Task : 1 - 2 Short

In [15]:
# Perform the 8-point algorithm to estimate the fundamental matrix F
F, _ = cv2.findFundamentalMat(points_image1_homogeneous, points_image2_homogeneous, cv2.FM_8POINT)

### Task 2 - 1

In [18]:
def linear_triangulation(point1, point2, P1, P2):
    A = np.zeros((4, 4))
    A[0] = point1[0] * P1[2] - P1[0]
    A[1] = point1[1] * P1[2] - P1[1]
    A[2] = point2[0] * P2[2] - P2[0]
    A[3] = point2[1] * P2[2] - P2[1]
    
    _, _, V = np.linalg.svd(A)
    X = V[-1, :3] / V[-1, 3]  # Homogeneous coordinates
    
    return X

In [ ]:
# Create the canonical camera pair
P1 = np.hstack((np.eye(3), np.zeros((3, 1))))
P2 = np.hstack((np.cross([0, 0, 1], [0, -1, 0]).reshape((3, 1)), [0, -1, 0]))

# Perform triangulation for each pair of corresponding points
X = []
for i in range(selected_points1.shape[0]):
    A = np.vstack((selected_points1[i, 0] * P1[2, :] - P1[0, :],
                   selected_points1[i, 1] * P1[2, :] - P1[1, :],
                   selected_points2[i, 0] * P2[2, :] - P2[0, :],
                   selected_points2[i, 1] * P2[2, :] - P2[1, :]))
    _, _, V = np.linalg.svd(A)
    X_homogeneous = V[-1, :4]
    X_homogeneous /= X_homogeneous[3]
    X.append(X_homogeneous[:3])

# Convert the triangulated points to numpy array
X = np.array(X)

# Save the triangulated points coordinates to a text file
np.savetxt("triangulated_points.txt", X, fmt="%.6f")

print("Triangulated points coordinates saved to 'triangulated_points.txt'.")


In [ ]:
# Create the canonical camera pair
P1 = np.hstack((np.eye(3), np.zeros((3, 1))))
P2 = np.hstack((np.cross([[0], [0], [1]], [[0], [-1], [0]]), [[0], [-1], [0]]))

# Perform triangulation for each pair of corresponding points
X = []
for i in range(selected_points1.shape[0]):
    A = np.vstack((selected_points1[i, 0] * P1[2, :] - P1[0, :],
                   selected_points1[i, 1] * P1[2, :] - P1[1, :],
                   selected_points2[i, 0] * P2[2, :] - P2[0, :],
                   selected_points2[i, 1] * P2[2, :] - P2[1, :]))
    _, _, V = np.linalg.svd(A)
    X_homogeneous = V[-1, :4]
    X_homogeneous /= X_homogeneous[3]
    X.append(X_homogeneous[:3])

# Convert the triangulated points to numpy array
X = np.array(X)

# Save the triangulated points coordinates to a text file
np.savetxt("triangulated_points.txt", X, fmt="%.6f")

In [19]:
# Create the canonical camera pair
P1 = np.hstack((np.eye(3), np.zeros((3, 1))))
P2 = np.hstack((np.cross([0, 0, 1], [0, -1, 0]).reshape((3, 1)), [0, -1, 0]))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [16]:
# Perform triangulation for each pair of corresponding points
world_coords = []
for i in range(len(points_image1)):
    point1 = normalized_points_image1[:, i]
    point2 = normalized_points_image2[:, i]
    X = linear_triangulation(point1, point2, P1, P2)
    world_coords.append(X)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
# Convert world coordinates to non-homogeneous form
world_coords = np.array(world_coords)[:, :3] / np.array(world_coords)[:, 3:]

# Print the reconstructed 3D coordinates
print("Reconstructed 3D Coordinates:")
for i, X in enumerate(world_coords):
    print("Point", i+1, ": ", X)

In [20]:
def normalize_camera_matrix(P):
    # Normalize the camera matrix by dividing each row by the norm of the row
    for i in range(3):
        P[i] /= np.linalg.norm(P[i, :3])
    return P

# Normalize the camera matrices
P1_normalized = normalize_camera_matrix(P1)
P2_normalized = normalize_camera_matrix(P2)

NameError: name 'P2' is not defined

In [ ]:
# Perform triangulation for each pair of corresponding points
world_coords = []
for i in range(len(points_image1)):
    point1 = normalized_points_image1[:, i]
    point2 = normalized_points_image2[:, i]
    X = linear_triangulation(point1, point2, P1_normalized, P2_normalized)
    world_coords.append(X)

# Convert world coordinates to non-homogeneous form
world_coords = np.array(world_coords)[:, :3] / np.array(world_coords)[:, 3:]

In [ ]:
# Print the reconstructed 3D coordinates
print("Reconstructed 3D Coordinates:")
for i, X in enumerate(world_coords):
    print("Point", i+1, ": ", X)

### Task 2 - 2 Short

In [ ]:
# Task 2: Reconstruct the projection matrices P and P' using the canonical camera pair

# Define the canonical camera matrices P_hat and P_hat_prime
P_hat = np.hstack((np.eye(3), np.zeros((3, 1))))
P_hat_prime = np.zeros((3, 4))
P_hat_prime[:,:3] = np.cross(P_hat[:3, 1:], F[:3, :3])
P_hat_prime[:,3] = np.squeeze(np.cross(P_hat[:3, :], P_hat_prime[:,:3]))

### Task 3 - 1

In [ ]:
def compute_homography(X_hat, X):
    assert len(X_hat) == len(X), "Number of points must match"
    assert len(X_hat) >= 4, "At least 4 points are required"

    num_points = len(X_hat)

    # Construct the matrix A for the DLT algorithm
    A = np.zeros((2 * num_points, 9))
    for i in range(num_points):
        X_hat_i = X_hat[i]
        X_i = X[i]
        A[2*i] = [0, 0, 0, -X_hat_i[0], -X_hat_i[1], -X_hat_i[2], X_i[1]*X_hat_i[0], X_i[1]*X_hat_i[1], X_i[1]*X_hat_i[2]]
        A[2*i+1] = [X_hat_i[0], X_hat_i[1], X_hat_i[2], 0, 0, 0, -X_i[0]*X_hat_i[0], -X_i[0]*X_hat_i[1], -X_i[0]*X_hat_i[2]]

    # Perform singular value decomposition of A
    _, _, V = np.linalg.svd(A)

    # Extract the homography H from the last column of V
    H = V[-1].reshape(3, 3)

    return H

# Compute the 3D homography H
H = compute_homography(world_coords, selected_world_coords)

# Print the homography matrix H
print("Homography Matrix H:")
print(H)


In [ ]:
def compute_homography(X_hat, X):
    assert len(X_hat) == len(X), "Number of points must match"
    assert len(X_hat) >= 4, "At least 4 points are required"

    num_points = len(X_hat)

    # Construct the matrix A for the DLT algorithm
    A = np.zeros((2 * num_points, 9))
    for i in range(num_points):
        X_hat_i = X_hat[i]
        X_i = X[i]
        A[2*i] = [0, 0, 0, -X_hat_i[0], -X_hat_i[1], -X_hat_i[2], X_i[1]*X_hat_i[0], X_i[1]*X_hat_i[1], X_i[1]*X_hat_i[2]]
        A[2*i+1] = [X_hat_i[0], X_hat_i[1], X_hat_i[2], 0, 0, 0, -X_i[0]*X_hat_i[0], -X_i[0]*X_hat_i[1], -X_i[0]*X_hat_i[2]]

    # Perform singular value decomposition of A
    _, _, V = np.linalg.svd(A)

    # Extract the homography H from the last column of V
    H = V[-1].reshape(3, 3)

    return H

# Compute the 3D homography H for each plane
num_planes = 2  # Number of planes (can be adjusted based on your scenario)
homographies = []

for i in range(num_planes):
    # Select the corresponding points for the current plane
    plane_indices = [0, 1, 2]  # Adjust the indices based on your scenario
    X_hat_plane = [selected_world_coords[j] for j in plane_indices]
    X_plane = [world_coords[j] for j in plane_indices]

    # Compute the homography for the current plane
    H_plane = compute_homography(X_hat_plane, X_plane)
    homographies.append(H_plane)

# Print the homography matrices for each plane
for i in range(num_planes):
    print(f"Homography Matrix for Plane {i+1}:")
    print(homographies[i])
    print()


### Task 3 - 2 Short

In [ ]:
# Task 3: Recover the 3D homography H using the DLT

# Load the world coordinates from "coords.txt"
world_coords = np.loadtxt("coords.txt")

In [ ]:
# Construct the matrix A for DLT
A = []
for i in range(len(world_coords)):
    x, y, z = world_coords[i]
    A.append([x, y, z, 1, 0, 0, 0, 0, -x * P_hat_prime[2, 0], -y * P_hat_prime[2, 1], -z * P_hat_prime[2, 2], -P_hat_prime[2, 3]])
    A.append([0, 0, 0, 0, x, y, z, 1, -x * P_hat_prime[1, 0], -y * P_hat_prime[1, 1], -z * P_hat_prime[1, 2], -P_hat_prime[1, 3]])
A = np.array(A)

In [ ]:
# Perform singular value decomposition (SVD) of A
_, _, V = np.linalg.svd(A)

In [ ]:
# Extract the last column of V to obtain the 3D homography H
H = V[-1].reshape((3, 4))

### Task 4 - 1

In [ ]:
import cv2
import numpy as np

# Load the images
image1 = cv2.imread('house1.png')
image2 = cv2.imread('house2.png')

# Load the coordinates of the user-marked points
user_points1 = np.loadtxt('user_points1.txt')
user_points2 = np.loadtxt('user_points2.txt')

# Perform the camera calibration and reconstruction steps as before to obtain the projection matrices P and P'
# ...

# Triangulation to find the world coordinates of the user-marked points
num_points = len(user_points1)
world_coords = np.zeros((num_points, 4))

for i in range(num_points):
    # Get the image coordinates of the user-marked points
    x1 = user_points1[i]
    x2 = user_points2[i]

    # Perform triangulation to compute the world coordinates
    A = np.zeros((4, 4))
    A[0] = x1[0] * P[2] - P[0]
    A[1] = x1[1] * P[2] - P[1]
    A[2] = x2[0] * P_prime[2] - P_prime[0]
    A[3] = x2[1] * P_prime[2] - P_prime[1]

    _, _, V = np.linalg.svd(A)
    X = V[-1, :3] / V[-1, 3]  # Normalize by the homogeneous coordinate

    world_coords[i] = X

# Display the world coordinates of the user-marked points
for i in range(num_points):
    x, y, z, _ = world_coords[i]
    print(f"World coordinates of point {i+1}: ({x}, {y}, {z})")

# Print the estimated values
print("Estimated Fundamental Matrix F:")
print(F)
print()

print("Estimated Projection Matrix P:")
print(P)
print()

print("Estimated Projection Matrix P':")
print(P_prime)
print()

### Task 4 - 2 Short

In [ ]:
# Task 4: Triangulation to find the world coordinates of the user-marked points

# Load the user-marked points from "user_points1.txt" and "user_points2.txt"
user_points1 = np.loadtxt("user_points1.txt")
user_points2 = np.loadtxt("user_points2.txt")

In [ ]:
# Perform triangulation to find the world coordinates
num_points = len(user_points1)
world_coords_estimated = np.zeros((num_points, 4))

In [ ]:
for i in range(num_points):
    x1 = user_points1[i]
    x2 = user_points2[i]

    # Construct the matrix A for DLT
    A = np.zeros((4, 4))
    A[0] = x1[0] * P_hat[2] - P_hat[0]
    A[1] = x1[1] * P_hat[2] - P_hat[1]
    A[2] = x2[0] * P_hat_prime[2] - P_hat_prime[0]
    A[3] = x2[1] * P_hat_prime[2] - P_hat_prime[1]

    # Perform singular value decomposition (SVD) of A
    _, _, V = np.linalg.svd(A)

    # Extract the last column of V to obtain the estimated world coordinates
    world_coords_estimated[i] = V[-1] / V[-1, 3]

In [ ]:
# Display the estimated world coordinates
print("Estimated World Coordinates (X_i'):")
print(world_coords_estimated)
print()

# Display the estimated Fundamental Matrix F
print("Estimated Fundamental Matrix F:")
print(F)
print()

# Display the estimated Projection Matrix P_hat
print("Estimated Projection Matrix P_hat:")
print(P_hat)
print()

# Display the estimated Projection Matrix P_hat_prime
print("Estimated Projection Matrix P_hat_prime:")
print(P_hat_prime)
print()

# Display the estimated 3D Homography H
print("Estimated 3D Homography H:")
print(H)
print()

In [ ]:
import cv2
import numpy as np



# Task 1: Reconstruct the fundamental matrix F using the 8-point algorithm

# Load the image points from "house1.png" and "house2.png"
image1 = cv2.imread("house1.png")
image2 = cv2.imread("house2.png")

# Load the image coordinates from the previous part
image_coords1 = np.loadtxt("image_coords1.txt")
image_coords2 = np.loadtxt("image_coords2.txt")

# Perform the 8-point algorithm to estimate the fundamental matrix F
F, _ = cv2.findFundamentalMat(image_coords1, image_coords2, cv2.FM_8POINT)



# Task 2: Reconstruct the projection matrices P and P' using the canonical camera pair

# Define the canonical camera matrices P_hat and P_hat_prime
P_hat = np.hstack((np.eye(3), np.zeros((3, 1))))
P_hat_prime = np.zeros((3, 4))
P_hat_prime[:,:3] = np.cross(P_hat[:3, 1:], F[:3, :3])
P_hat_prime[:,3] = np.squeeze(np.cross(P_hat[:3, :], P_hat_prime[:,:3]))



# Task 3: Recover the 3D homography H using the DLT

# Load the world coordinates from "coords.txt"
world_coords = np.loadtxt("coords.txt")

# Construct the matrix A for DLT
A = []
for i in range(len(world_coords)):
    x, y, z = world_coords[i]
    A.append([x, y, z, 1, 0, 0, 0, 0, -x * P_hat_prime[2, 0], -y * P_hat_prime[2, 1], -z * P_hat_prime[2, 2], -P_hat_prime[2, 3]])
    A.append([0, 0, 0, 0, x, y, z, 1, -x * P_hat_prime[1, 0], -y * P_hat_prime[1, 1], -z * P_hat_prime[1, 2], -P_hat_prime[1, 3]])
A = np.array(A)

# Perform singular value decomposition (SVD) of A
_, _, V = np.linalg.svd(A)

# Extract the last column of V to obtain the 3D homography H
H = V[-1].reshape((3, 4))



# Task 4: Triangulation to find the world coordinates of the user-marked points

# Load the user-marked points from "user_points1.txt" and "user_points2.txt"
user_points1 = np.loadtxt("user_points1.txt")
user_points2 = np.loadtxt("user_points2.txt")

# Perform triangulation to find the world coordinates
num_points = len(user_points1)
world_coords_estimated = np.zeros((num_points, 4))

for i in range(num_points):
    x1 = user_points1[i]
    x2 = user_points2[i]

    # Construct the matrix A for DLT
    A = np.zeros((4, 4))
    A[0] = x1[0] * P_hat[2] - P_hat[0]
    A[1] = x1[1] * P_hat[2] - P_hat[1]
    A[2] = x2[0] * P_hat_prime[2] - P_hat_prime[0]
    A[3] = x2[1] * P_hat_prime[2] - P_hat_prime[1]

    # Perform singular value decomposition (SVD) of A
    _, _, V = np.linalg.svd(A)

    # Extract the last column of V to obtain the estimated world coordinates
    world_coords_estimated[i] = V[-1] / V[-1, 3]

# Display the estimated world coordinates
print("Estimated World Coordinates (X_i'):")
print(world_coords_estimated)
print()

# Display the estimated Fundamental Matrix F
print("Estimated Fundamental Matrix F:")
print(F)
print()

# Display the estimated Projection Matrix P_hat
print("Estimated Projection Matrix P_hat:")
print(P_hat)
print()

# Display the estimated Projection Matrix P_hat_prime
print("Estimated Projection Matrix P_hat_prime:")
print(P_hat_prime)
print()

# Display the estimated 3D Homography H
print("Estimated 3D Homography H:")
print(H)
print()


In [21]:

# Load the images
image1 = cv2.imread("house1.png")
image2 = cv2.imread("house2.png")

# Display the images
cv2.imshow("Image 1", image1)
cv2.imshow("Image 2", image2)

# Wait for a key press
cv2.waitKey(0)
cv2.destroyAllWindows()

# Define the corresponding points coordinates in image 1 (replace with actual coordinates)
points1 = np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4], [x5, y5], [x6, y6], [x7, y7], [x8, y8], [x9, y9], [x10, y10]])

# Define the corresponding points coordinates in image 2 (replace with actual coordinates)
points2 = np.array([[x1_prime, y1_prime], [x2_prime, y2_prime], [x3_prime, y3_prime], [x4_prime, y4_prime], [x5_prime, y5_prime], [x6_prime, y6_prime], [x7_prime, y7_prime], [x8_prime, y8_prime], [x9_prime, y9_prime], [x10_prime, y10_prime]])

# Perform the 8-point algorithm to estimate the fundamental matrix
F, _ = cv2.findFundamentalMat(points1, points2, cv2.FM_8POINT)

# Create the canonical camera pair
P_hat = np.hstack((np.eye(3), np.zeros((3, 1))))
e_prime_cross = np.dot(F, np.eye(3))
P_hat_prime = np.hstack((e_prime_cross, e_prime_cross))

# Perform triangulation for each pair of corresponding points
X_hat = []
for i in range(points1.shape[0]):
    A = np.vstack((points1[i, 0] * P_hat[2, :] - P_hat[0, :],
                   points1[i, 1] * P_hat[2, :] - P_hat[1, :],
                   points2[i, 0] * P_hat_prime[2, :] - P_hat_prime[0, :],
                   points2[i, 1] * P_hat_prime[2, :] - P_hat_prime[1, :]))
    _, _, V = np.linalg.svd(A)
    X_homogeneous = V[-1, :4]
    X_homogeneous /= X_homogeneous[3]
    X_hat.append(X_homogeneous[:3])

# Convert the triangulated points to numpy array
X_hat = np.array(X_hat)

# Save the fundamental matrix and triangulated points coordinates
np.savetxt("fundamental_matrix.txt", F)
np.savetxt("triangulated_points.txt", X_hat)

print("Fundamental matrix and triangulated points coordinates saved.")


NameError: name 'x3' is not defined

In [25]:
def construct_P_matrix(camera_matrix, rotation_matrix, translation_vector):
    P_matrix = np.dot(camera_matrix, np.hstack((rotation_matrix, translation_vector)))
    return P_matrix


def linear_triangulation(points1, points2, camera_matrix1, camera_matrix2):
    num_points = points1.shape[0]
    points_3d = np.zeros((num_points, 3))

    for i in range(num_points):
        x1 = points1[i, 0]
        y1 = points1[i, 1]
        x2 = points2[i, 0]
        y2 = points2[i, 1]

        A = np.zeros((4, 4))
        A[0, :] = x1 * camera_matrix1[2, :] - camera_matrix1[0, :]
        A[1, :] = y1 * camera_matrix1[2, :] - camera_matrix1[1, :]
        A[2, :] = x2 * camera_matrix2[2, :] - camera_matrix2[0, :]
        A[3, :] = y2 * camera_matrix2[2, :] - camera_matrix2[1, :]

        _, _, V = np.linalg.svd(A)
        point_3d_homogeneous = V[-1, :]
        point_3d = point_3d_homogeneous[:3] / point_3d_homogeneous[3]
        points_3d[i, :] = point_3d

    return points_3d


# Load the images
image1 = cv2.imread('house1.png')
image2 = cv2.imread('house2.png')

# Define the image coordinates of the corresponding points
points1 = np.array([[x1, y1] for x1, y1 in [(100, 200), (300, 250), (400, 150), (250, 400), (150, 500),
                                            (600, 300), (700, 400), (500, 100), (400, 300), (200, 600)]])
points2 = np.array([[x2, y2] for x2, y2 in [(80, 180), (280, 220), (390, 130), (230, 380), (120, 480),
                                            (620, 270), (720, 380), (480, 90), (370, 280), (190, 580)]])

# Normalize image coordinates (mean 0, variance 1)
points1_normalized = cv2.normalize(points1.astype('float'), None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
points2_normalized = cv2.normalize(points2.astype('float'), None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

# Estimate the fundamental matrix using the 8-point algorithm
fundamental_matrix, _ = cv2.findFundamentalMat(points1_normalized, points2_normalized, cv2.FM_8POINT)

# Construct the projection matrices for the canonical camera pair
camera_matrix1 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
camera_matrix2 = np.zeros((3, 3))
rotation_matrix2 = np.zeros((3, 3))
translation_vector2 = np.zeros((3, 1))

# Compute the essential matrix
essential_matrix = np.dot(np.dot(camera_matrix2.T, fundamental_matrix), camera_matrix1)

# Perform SVD decomposition of the essential matrix
# _, rotation_matrix1, translation_vector1, rotation_matrix2, translation_vector2 = cv2.decomposeEssentialMat(essential_matrix)

################################################################################################################################
# _, rotation_matrix, translation_vector, _, _ = cv2.decomposeEssentialMat(essential_matrix)

# # Separate the translation vector into x, y, z components
# tx = translation_vector[0]
# ty = translation_vector[1]
# tz = translation_vector[2]

################################################################################################################################
_, rotation_matrix, translation_vector, _ = cv2.recoverPose(essential_matrix, points1_normalized, points2_normalized)

# Separate the translation vector into x, y, z components
tx = translation_vector[0]
ty = translation_vector[1]
tz = translation_vector[2]


# Construct the projection matrices
# projection_matrix1 = construct_P_matrix(camera_matrix1, rotation_matrix1, translation_vector1)
# projection_matrix2 = construct_P_matrix(camera_matrix2, rotation_matrix2, translation_vector2)

####################################################################################################################################
# Construct the projection matrices
projection_matrix1 = construct_P_matrix(camera_matrix1, np.eye(3), np.zeros((3, 1)))
projection_matrix2 = construct_P_matrix(camera_matrix2, rotation_matrix, translation_vector)


# Perform linear triangulation
points_3d = linear_triangulation(points1_normalized, points2_normalized, projection_matrix1, projection_matrix2)

print("Fundamental Matrix:")
print(fundamental_matrix)
print("\nProjection Matrix 1:")
print(projection_matrix1)
print("\nProjection Matrix 2:")
print(projection_matrix2)
print("\n3D Points:")
print(points_3d)


Fundamental Matrix:
[[-0.07470452 -2.24339246  3.95327814]
 [ 2.04283486 -0.44346621 10.19184432]
 [-3.89917458 -9.83698603  1.        ]]

Projection Matrix 1:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]

Projection Matrix 2:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

3D Points:
[[ inf  inf -inf]
 [-inf -inf  inf]
 [-inf -inf  inf]
 [-inf -inf  inf]
 [ inf -inf -inf]
 [ inf -inf  inf]
 [ inf -inf  inf]
 [ inf -inf  inf]
 [-inf -inf  inf]
 [ inf -inf -inf]]


C:\Users\admin\AppData\Local\Temp/ipykernel_2696/3430552686.py:24: RuntimeWarning: divide by zero encountered in divide
  point_3d = point_3d_homogeneous[:3] / point_3d_homogeneous[3]


In [26]:
import cv2
import numpy as np

def find_image_coordinates(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Create a window and display the image for manual selection
    cv2.namedWindow("Image")
    cv2.imshow("Image", image)
    cv2.waitKey(0)

    # Initialize an empty list to store the image coordinates
    image_points = []

    # Mouse callback function for manual selection
    def select_point(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            # Append the image coordinates of the selected point to the list
            image_points.append((x, y))
            # Draw a red dot at the selected point
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
            # Display the updated image
            cv2.imshow("Image", image)

    # Register the mouse callback function
    cv2.setMouseCallback("Image", select_point)

    # Wait for the user to close the image window
    cv2.waitKey(0)

    # Close all OpenCV windows
    cv2.destroyAllWindows()

    return image_points

def estimate_fundamental_matrix(image_points1, image_points2):
    # Convert image points to homogeneous coordinates
    image_points1_homo = np.hstack((image_points1, np.ones((len(image_points1), 1))))
    image_points2_homo = np.hstack((image_points2, np.ones((len(image_points2), 1))))

    # Normalize image points
    image_points1_normalized = cv2.normalize(image_points1_homo, None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    image_points2_normalized = cv2.normalize(image_points2_homo, None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Estimate fundamental matrix using normalized eight-point algorithm
    F, _ = cv2.findFundamentalMat(image_points1_normalized, image_points2_normalized, cv2.FM_8POINT)

    return F


def estimate_homography(image_points1, image_points2):
    # Construct the matrix A
    num_points = image_points1.shape[0]
    A = np.zeros((2*num_points, 9))
    for i in range(num_points):
        x1, y1 = image_points1[i]
        x2, y2 = image_points2[i]
        A[2*i] = [-x1, -y1, -1, 0, 0, 0, x2*x1, x2*y1, x2]
        A[2*i+1] = [0, 0, 0, -x1, -y1, -1, y2*x1, y2*y1, y2]

    # Perform singular value decomposition
    _, _, V = np.linalg.svd(A)

    # Extract the homography matrix H from the last column of V
    H = V[-1].reshape(3, 3)

    return H

def triangulate_points(image_points1, image_points2, K1, K2, R1, R2, Ce1, Ce2):
    num_points = image_points1.shape[0]
    points_3d = np.zeros((num_points, 3))

    # Camera matrices for the camera pair
    P = K1 @ np.hstack((R1, -R1 @ Ce1.reshape(3, 1)))
    P_prime = K2 @ np.hstack((R2, -R2 @ Ce2.reshape(3, 1)))

    for i in range(num_points):
        x1, y1 = image_points1[i]
        x2, y2 = image_points2[i]

        # Homogeneous image coordinates
        x1_homo = np.array([[x1], [y1], [1]])
        x2_homo = np.array([[x2], [y2], [1]])

        # Linear triangulation
        A = np.zeros((4, 4))
        A[0] = x1_homo[0] * P[2] - P[0]
        A[1] = x1_homo[1] * P[2] - P[1]
        A[2] = x2_homo[0] * P_prime[2] - P_prime[0]
        A[3] = x2_homo[1] * P_prime[2] - P_prime[1]

        _, _, V = np.linalg.svd(A)
        X_homo = V[-1]
        X_homo /= X_homo[3]  # Normalize by the fourth coordinate

        # Convert homogeneous coordinates to 3D coordinates
        X = X_homo[:3]

        # Store the 3D point
        points_3d[i] = X

    return points_3d

def main():
    # Task 1: Get the image coordinates of the points from both images
    image_path1 = "house1.png"  # Replace with the actual path to the first image
    image_path2 = "house2.png"  # Replace with the actual path to the second image

    image_points1 = find_image_coordinates(image_path1)
    image_points2 = find_image_coordinates(image_path2)

    # Task 2: Reconstruct the fundamental matrix and triangulate 3D points
    F = estimate_fundamental_matrix(image_points1, image_points2)

    # Assuming the camera matrices and camera centers for the first image are known
    K1 = np.array([[focal_length, 0, principal_point_x],
                   [0, focal_length, principal_point_y],
                   [0, 0, 1]])  # Replace with the actual values
    R1 = np.eye(3)  # Replace with the actual rotation matrix
    Ce1 = np.array([0, 0, 0])  # Replace with the actual camera center

    # Assuming the camera matrices and camera centers for the second image are known
    K2 = np.array([[focal_length, 0, principal_point_x],
                   [0, focal_length, principal_point_y],
                   [0, 0, 1]])  # Replace with the actual values
    R2 = np.eye(3)  # Replace with the actual rotation matrix
    Ce2 = np.array([baseline, 0, 0])  # Replace with the actual camera center

    # Triangulate the 3D points
    points_3d = triangulate_points(image_points1, image_points2, K1, K2, R1, R2, Ce1, Ce2)

    # Task 3: Recover the 3D homography H
    # Assuming the camera matrices and camera centers for the canonical camera pair are known
    K_hat = np.eye(3)  # Replace with the actual calibration matrix
    R_hat = np.eye(3)  # Replace with the actual rotation matrix
    Ce_hat = np.zeros(3)  # Replace with the actual camera center

    H = estimate_homography(points_3d, coords_world)

    # Task 4: Triangulate the points using the correct camera pair
    points_3d_correct = triangulate_points(image_points1, image_points2, K1, K2, R1, R2, Ce1, Ce2)

    # Print the 3D coordinates of the points
    print("Reconstructed 3D Coordinates (with canonical camera pair):")
    for i in range(points_3d.shape[0]):
        X = points_3d[i]
        print("Point {}: {}".format(i+1, X))

    print("\nReconstructed 3D Coordinates (with correct camera pair):")
    for i in range(points_3d_correct.shape[0]):
        X = points_3d_correct[i]
        print("Point {}: {}".format(i+1, X))

if __name__ == "__main__":
    main()


NameError: name 'focal_length' is not defined

In [ ]:
import matplotlib.pyplot as plt

def visualize_image(image_path, image_points):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create a figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image_rgb)

    # Plot the image points
    for point in image_points:
        x, y = point
        ax.plot(x, y, 'ro', markersize=5)

    # Set axis labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

    # Set title
    ax.set_title('Image with Marked Points')

    # Show the plot
    plt.show()

# Example usage
image_path = "house1.png"  # Replace with the actual path to the image
image_points = find_image_coordinates(image_path)  # Replace with your code to find image coordinates
visualize_image(image_path, image_points)